<a href="https://colab.research.google.com/github/Chuc-ngan/Low-Light-Image-Enhancement/blob/main/HE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==2.18.0  # Thay đổi phiên bản theo nhu cầu

# **1. Kết nối với Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/MyDrive/ColabNotebooks/TieuLuan'

!pwd

Mounted at /content/gdrive
/content/gdrive/.shortcut-targets-by-id/1DnsWp1mopgehQJUYpN4ERfnj2CyapJGW/ColabNotebooks/TieuLuan
/content/gdrive/.shortcut-targets-by-id/1DnsWp1mopgehQJUYpN4ERfnj2CyapJGW/ColabNotebooks/TieuLuan


# **2. Import thư viện**

In [ ]:
# ===================== 1. Xử lý dữ liệu =====================
import os
import shutil  # Quản lý tệp/thư mục
import random
import math
import collections
import numpy as np  # Xử lý số học
import pandas as pd  # Xử lý dữ liệu CSV
from pathlib import Path
from PIL import Image, UnidentifiedImageError
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
import pickle
# ===================== 2. Machine Learning =====================
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import seaborn as sns

# ===================== 3. TensorFlow & Keras =====================
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import mixed_precision
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau

# ===================== 4. Xây dựng mô hình =====================
from tensorflow.keras.layers import (
    Input, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization,
    GlobalAveragePooling2D, Dense, Dropout, RepeatVector, Reshape, Flatten
)
from tensorflow.keras.applications import EfficientNetB0, ResNet50
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input, decode_predictions
from keras.layers import add, concatenate
from keras import layers, backend as K

# ===================== 5. Xử lý hình ảnh & Vẽ biểu đồ =====================
import cv2 as cv
import matplotlib.pyplot as plt

# ===================== 6. Đặt Seed (Reproducibility) =====================
tf.random.set_seed(42)  # Thay số 42 bằng giá trị seed mong muốn
np.random.seed(1)


# **3. Cấu hình môi trường TensorFlow**

In [ ]:
# Thiết lập GPU sử dụng bộ nhớ động (tránh lỗi thiếu bộ nhớ)
gpu_devices = tf.config.list_physical_devices('GPU')
if gpu_devices:
    tf.config.experimental.set_memory_growth(gpu_devices[0], True)

# Kiểm tra TensorFlow đang chạy trên GPU hay không
print("TensorFlow is running on GPU:", tf.test.is_built_with_cuda())
# Bật chế độ Mixed Precision để tăng tốc (nếu GPU hỗ trợ)
# Thiết lập mixed precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

# Bật XLA với cấu hình tối ưu
tf.config.optimizer.set_jit(True)
tf.config.optimizer.set_experimental_options({
    "layout_optimizer": True,
    "constant_folding": True,
    "shape_optimization": True,
    "remapping": True,
    "arithmetic_optimization": True,
    "dependency_optimization": True,
    "loop_optimization": True,
    "function_optimization": True,
    "debug_stripper": True
})

TensorFlow is running on GPU: True


# **4. Configuration**

In [ ]:
# Thư mục thiếu sáng
low_light_dir = "dataset/low_light"

low_light_noise_dir = "dataset/low_light_noise"

# Thư mục chứa ảnh sau khi tăng cường
enhanced_image_low_light_dir = "dataset/enhanced_he_low_light"
enhanced_image_low_light_noise_dir = "dataset/enhanced_he_low_light_noise"

# **HE**

In [ ]:
def enhance_with_clahe(input_dir, output_dir, img_size=(224, 224)):
    os.makedirs(output_dir, exist_ok=True)

    all_images = []
    for root, _, files in os.walk(input_dir):
        rel_path = os.path.relpath(root, input_dir)
        save_dir = os.path.join(output_dir, rel_path)
        os.makedirs(save_dir, exist_ok=True)

        for file in files:
            if file.lower().endswith((".jpg", ".png", ".jpeg")):
                all_images.append((os.path.join(root, file), os.path.join(save_dir, file)))

    clahe = cv.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))

    with tqdm(total=len(all_images), desc="Enhancing with CLAHE", ncols=100) as pbar:
        for img_path, save_path in all_images:
            img = cv.imread(img_path)
            img = cv.resize(img, img_size)

            # Chuyển sang không gian màu YCrCb để chỉ xử lý độ sáng (kênh Y)
            ycrcb = cv.cvtColor(img, cv.COLOR_BGR2YCrCb)
            y, cr, cb = cv.split(ycrcb)
            y_eq = clahe.apply(y)
            merged = cv.merge([y_eq, cr, cb])
            result = cv.cvtColor(merged, cv.COLOR_YCrCb2BGR)

            cv.imwrite(save_path, result)
            pbar.update(1)

    print(f"✅ Đã lưu {len(all_images)} ảnh sau khi áp dụng CLAHE vào: {output_dir}")

In [ ]:
enhance_with_clahe(low_light_dir, enhanced_image_low_light_dir, img_size=(224, 224))

Enhancing with CLAHE: 100%|█████████████████████████████████| 10402/10402 [1:35:23<00:00,  1.82it/s]

✅ Đã lưu 10402 ảnh sau khi áp dụng CLAHE vào: dataset/enhanced_he_low_light


In [ ]:
enhance_with_clahe(low_light_noise_dir, enhanced_image_low_light_noise_dir, img_size=(224, 224))

Enhancing with CLAHE: 100%|███████████████████████████████████| 10402/10402 [32:19<00:00,  5.36it/s]

✅ Đã lưu 10402 ảnh sau khi áp dụng CLAHE vào: dataset/enhanced_he_low_light_noise
